# モンテカルロシミュレーション入門

モンテカルロシミュレーションを題材にPythonの練習をしましょう。

モンテカルロシミュレーションとは、名前はなんだか格好良い（しかも難しそう）ですが、中身はとても簡単です。
ただ、乱数を発生させて、その乱数を使って計算しようというのがコンセプトです。

# 演習： 半径3の円の面積を求めましょう。
ヒント：  $ x^2 + y^2 = 3^2 $

---

#### 考え方: 
縦、横の3の正方形を思い浮かべてください。その中に、求めたい円の4分の1が入っています。<br>
モンテカルロシミュレーションで求める場合、縦方向・横方向それぞれで0~3の乱数を発生させて、円の4分の1に入る割合を調べます。<br>
割合を求めたら、正方形の面積にその割合をかけて4倍すれば面積は推定できます。<br>

---

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt  #matplotlibはグラフ描画のためのライブラリ

## ランダムに値(x,y)を生成する関数

In [ ]:
#sizeはサンプル数
def generate_uniform_random(size):
    x = np.random.rand(size)*3
    y = np.random.rand(size)*3
    return x,y

In [ ]:
x,y = generate_uniform_random(1000)
print(x[0:10])
print(y[0:10])

どんな点が生成されているのかを可視化しましょう

In [ ]:
plt.figure(figsize=(3, 3)) # 適当に縦横のサイズを合わせる
plt.scatter(x, y)  #　resの1列目をx軸に、resの2列目をy軸にして散布図(scatter plot)に
plt.show()

続いて、例題の円の公式  ($x^2 + y^2 = 3^2$)を参考に、円の中に入ったら1, 円の外の場合は0になる関数を書きましょう。

In [ ]:
def in_circle_func(x, y):
    if x**2 + y**2 <= 3**2:
        return 1
    else:
        return 0

In [ ]:
print(in_circle_func(x=1, y=1))
print(in_circle_func(x=4, y=3))

続いて、生成した乱数のうち円の中に入っている割合を調べてみましょう

In [ ]:
in_circle_count = 0
in_circle_data_x = []
in_circle_data_y = []
out_circle_data_x = []
out_circle_data_y = []

for i in range(1000):
    in_circle_flag = in_circle_func(x[i], y[i])  #もし円の中であれば1, そうでなければ0
    in_circle_count += in_circle_flag #足し上げていけば円の中に入る点の数を数えられる
    
    #あとで色分けした散布図を書くためにデータを分けておく
    if in_circle_flag==1:
        in_circle_data_x.append(x[i])
        in_circle_data_y.append(y[i])
    else:
        out_circle_data_x.append(x[i])
        out_circle_data_y.append(y[i])

in_circle_ratio = in_circle_count/len(x)
print("円の中に入る割合",  in_circle_ratio)

上記の流れを関数にしておきましょう

In [ ]:
def get_incircle_ratio(x, y):
    in_circle_count = 0
    in_circle_data_x = []
    in_circle_data_y = []
    out_circle_data_x = []
    out_circle_data_y = []

    for i in range(len(x)):
        in_circle_flag = in_circle_func(x[i], y[i])  #もし円の中であれば1, そうでなければ0
        in_circle_count += in_circle_flag #足し上げていけば円の中に入る点の数を数えられる

    in_circle_ratio = in_circle_count/len(x)
    return in_circle_ratio

データを可視化してみましょう（以下のセルに記載されているコードを理解する必要はありません）

In [ ]:
fig = plt.figure(figsize=[5,5])
plt.scatter(in_circle_data_x,in_circle_data_y, c='red')
plt.scatter(out_circle_data_x,out_circle_data_y, c='blue')
plt.show()

乱数の発生エリアは、横軸(x)は0~3、縦軸(y)も0~3です。そのため、計算した円の中に入る割合（正確には扇型の中に入る割合）は3 x 3 = 9の面積の中の話です。そのため、3 x 3 x 円の中に入る割合 x 4 で円の面積が求められます。

In [ ]:
estimated_are = 3*3*in_circle_ratio*4
print("推定された値", estimated_are)
print("答え", 3*3*np.pi)

# モンテカルロシミュレーションではサンプル数が推定の精度に関係するので、サンプル数を変えながら、精度を見ていこう

In [ ]:
estimated_area = []
sample_size_list = np.arange(100,20000,100)  #サンプル数を100から20000まで、100刻みでリストとして用意

for sample_size in sample_size_list:
    
    # 乱数を生成
    rand_nums_x, rand_nums_y = generate_uniform_random(size=sample_size)

    # 面積の計算
    result = (3*3)*4*get_incircle_ratio(rand_nums_x, rand_nums_y)

    #後で可視化するために結果をリストに保存
    estimated_area.append(result)

サンプル数（x軸）を増やしたときの推定面積（y軸）を可視化してみましょう。だんだんと安定していっていることがわかります。

In [ ]:
# 横軸をsample_size, 縦軸にestimated_area として折れ線グラフを作成
plt.plot(sample_size_list, estimated_area)
plt.show()

In [ ]:
print("推定された値", estimated_area[-1])  #サンプルサイズが最大のときの推定された面積
print("答え", 3*3*np.pi)